In [15]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification
import torch







# Load the CSV file into a DataFrame
path = "PREPARE_Datasets_Hatebase/"


#file_path = path + "FINAL_IMSyPP_EN_644_PREPROCESSED_no_restricted_videos_features_2ktoken_w_prompt_u_desc_audio_object.xlsx" #path + "IMSyPP_EN_YouTube_comments_evaluation_context_PREPROCESSED_no_restricted_videos_1000.csv" #'stitch_mit_dataset_example.csv  # Update this path to the actual file location

file_path = "used_rows_for_evaluation_with_GPT35T.xlsx" #path + "FINAL_IMSyPP_EN_561_PREPROCESSED_no_restricted_videos_features_2ktoken_w_prompt_u_desc_audio_object_with_video_and_reply_metadata_no_missing.xlsx" #path + "IMSyPP_EN_YouTube_comments_evaluation_context_PREPROCESSED_no_restricted_videos_1000.csv" #'stitch_mit_dataset_example.csv  # Update this path to the actual file location

#file_path = "used_rows_for_evaluation_1517_FINAL.csv"

df = pd.read_excel(file_path)
df_gt = df.copy()
len(df)

222

In [5]:


df['type_reply'].value_counts()


type_reply
appropriate      928
offensive        273
inappropriate      8
violent            3
Name: count, dtype: int64

In [6]:
# Load the model and tokenizer
model_name = "IMSyPP/hate_speech_en"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Make sure the model is in evaluation mode
model.eval()

# Function for classifying a single text input
def classify_text(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    
    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the predicted class label
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    
    return predicted_class

In [7]:
# Classify each text in the 'reply' column
df['predicted_class'] = df['reply'].apply(classify_text)
df#.head()



,Unnamed: 0,video_id,comment_id,target,annotator,...,type_reply,target_reply,reply_id,target.1,predicted_class
0,0,NaN,Ugw3h-Rft03GK0sQyoB4AaABAg,NaN,HA,...,appropriate,NaN,Ugw3h-Rft03GK0sQyoB4AaABAg.98MssOlE_zD98YOv6q7jxC,NaN,0
1,2,NaN,UgwcIWd1Fw4UGiyHQZl4AaABAg,NaN,HA,...,offensive,12. other,UgwcIWd1Fw4UGiyHQZl4AaABAg.98VmXXgKlbv98h-fsKmNVE,NaN,0
2,3,NaN,UgwGH3n9-kMziH6MHJJ4AaABAg,5. religion,HA,...,appropriate,NaN,UgwGH3n9-kMziH6MHJJ4AaABAg.98eCDJk381J98eCyDSbc_6,5. religion,2
3,5,NaN,UgwL9uf1fl_LfsuMcxt4AaABAg,NaN,HA,...,offensive,4. antisemitism,UgwL9uf1fl_LfsuMcxt4AaABAg.98_JpvVTnup98xY_oefc7G,NaN,2
4,6,NaN,UgwlmDegdyuISJMwadp4AaABAg,NaN,HA,...,offensive,4. antisemitism,UgwlmDegdyuISJMwadp4AaABAg.98PgBjc6PgS98tEgARWsAO,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...
1207,1510,98zqHfGhM7A,UgzbueTCliJIBSJbvbd4AaABAg,NaN,PP,...,appropriate,NaN,UgzbueTCliJIBSJbvbd4AaABAg.98E1WpTbw6998E6OZoCtCN,NaN,0
1208,1512,98zqHfGhM7A,UgzOTAFni3o7UW1b71p4AaABAg,10. politics,PP,...,offensive,11. individual,UgzOTAFni3o7UW1b71p4AaABAg.98DyKQJ4Vgq98V1kvB0-nY,10. politics,2
1209,1513,98zqHfGhM7A,UgzTeEAW3ZrQ2c4dwDZ4AaABAg,10. politics,PP,...,offensive,10. politics,UgzTeEAW3ZrQ2c4dwDZ4AaABAg.98EHx_DfZHC98Hz4iLaiyn,10. politics,2
1210,1514,98zqHfGhM7A,UgztSHHRdIBAVga82yJ4AaABAg,NaN,PP,...,appropriate,NaN,UgztSHHRdIBAVga82yJ4AaABAg.98FgQdv8t5r98FldH9ZVls,NaN,0


In [8]:
df["type_reply"].value_counts()

type_reply
appropriate      928
offensive        273
inappropriate      8
violent            3
Name: count, dtype: int64

In [9]:
# Ensure a safe copy of the DataFrame
df_safe = df.copy()

# Mapping for new class names
class_mapping = {
"appropriate": 0,
"inappropriate": 1,
"offensive": 2,
"violent": 3
}

# Map class names to numeric labels
df_safe['type_reply'] = df_safe['type_reply'].map(class_mapping)

# Convert the true and predicted labels to lists
y_true = df_safe['type_reply'].tolist()

In [12]:
#df_safe['type_reply'].value_counts()

type_reply
0    928
2    273
1      8
3      3
Name: count, dtype: int64

In [13]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_metrics(df):
    # Ensure a safe copy of the DataFrame
    df_safe = df.copy()
    
    # Mapping for new class names
    class_mapping = {
        "appropriate": 0,
        "inappropriate": 1,
        "offensive": 2,
        "violent": 3
    }

    # Map class names to numeric labels
    df_safe['type_reply'] = df_safe['type_reply'].map(class_mapping)
    
    # Convert the true and predicted labels to lists
    y_true = df_safe['type_reply'].tolist()
    y_pred = df_safe['predicted_class'].tolist()
    
    # Ensure the lengths match
    if len(y_pred) != len(y_true):
        raise ValueError("The length of predicted and true labels does not match.")
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)
    
    # Calculate precision, recall, and f1-score in macro mode
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    
    # Create a DataFrame to display the results horizontally
    metrics_df = pd.DataFrame([{
        'Accuracy': accuracy,
        'Precision (Macro)': precision,
        'Recall (Macro)': recall,
        'F1 Score (Macro)': f1
    }])

    # Display the DataFrame
    return metrics_df

# Example usage:
# Assuming df is your DataFrame with 'type_reply' and 'predicted_class' columns
metrics_df = calculate_metrics(df)
#print(metrics_df)


In [14]:
metrics_df

,Accuracy,Precision (Macro),Recall (Macro),F1 Score (Macro)
0,0.833333,0.511356,0.437468,0.463262


In [ ]:
classwise_metrics


NameError: name 'classwise_metrics' is not defined

In [31]:
overall_metrics

NameError: name 'overall_metrics' is not defined

predicted_class
0    270
2     41
3      1
Name: count, dtype: int64

'offensive'

In [21]:
df['type_reply']

0      0
1      2
2      2
3      0
4      0
      ..
307    0
308    0
309    0
310    0
311    0
Name: type_reply, Length: 312, dtype: int64

312
312


Unique classes and their names:
[0, 2, 3]
['appropriate', 'offensive', 'violent']
     Unnamed: 0.1  Unnamed: 0     video_id                  comment_id  \
0              31          31  4MfW_BrHwOU  Ugwnv8UnkM97dT4AxS54AaABAg   
1              32          32  4MfW_BrHwOU  Ugx6PwrVvjxHuGK4PJd4AaABAg   
2              33          33  4MfW_BrHwOU  UgxHHeuiFiyf82CxDjR4AaABAg   
3              36          36  4MfW_BrHwOU  Ugyd1my4ilM4XnCemnZ4AaABAg   
4              38          38  4MfW_BrHwOU  UgyvJng1wWbMDgRrR4Z4AaABAg   
..            ...         ...          ...                         ...   
307          1487        1487  SwWFhzgXgsk  Ugy2p57CwYiJNrcaMUl4AaABAg   
308          1489        1489  SwWFhzgXgsk  Ugyb5TqkSJ-guP0RJc14AaABAg   
309          1490        1490  SwWFhzgXgsk  UgyoZ1rl9hJ2coz333h4AaABAg   
310          1492        1492  SwWFhzgXgsk  Ugz_BzNvWskEZ6yWxXx4AaABAg   
311          1493        1493  SwWFhzgXgsk  Ugz3cNWoftrVFqWL51R4AaABAg   

                             

c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Class,Precision,Recall,F1 Score,Support
0,appropriate,0.925926,0.936330,0.931099,267
1,offensive,0.609756,0.555556,0.581395,45
2,violent,0.000000,0.000000,0.000000,0


,Metric,Value
0,Accuracy,0.881410
1,Precision,0.511894
2,Recall,0.497295
3,F1 Score,0.504165
